# AAVE v3 Mainnet

**What is Aave?**

Aave is a decentralized finance protocol that allows people to lend and borrow cryptocurrencies. 
Lenders earn interest by depositing digital assets into specially created liquidity pools. Borrowers can then use their crypto as collateral to take out a flash loan using this liquidity.

**What does this example cover?**

In this example, we utilize the Subgrounds library to interact with the AAVE v3 Ethereum subgraph and fetch data related to its various usage metrics and lending protocols. This example explains how to load a subgraph, query it, handle and transform the data returned, and even visualize it. The detailed breakdown is as follows:

- Importing Libraries: The subgrounds library is imported for handling decentralized subgraphs, and pandas is used for handling data manipulation.

- Subgrounds Initialization: A Subgrounds object is instantiated using the `Playgrounds API key`.

- Loading AAVE v3 Subgraph: The AAVE v3 subgraph is loaded by passing its ID to a URL string and calling `sg.load_subgraph()`.

- Querying Subgraph: A `UsageMetricsDailySnapshot` query is performed on the subgraph, where the first 10 entries are fetched in descending order.

- Data Extraction: The extracted data, including `timestamp`, `daily active borrowers`, and `daily active depositors`, are fetched into a DataFrame using `sg.query_df()`.

- Converting Timestamps: Two methods are demonstrated for converting timestamp data to a more human-readable format using the `SyntheticField` object and the `datetime.fromtimestamp()` function.

- Querying Lending Protocols: Queries are performed on the `lendingProtocols` entity of the AAVE v3 subgraph, fetching all fields and specific fields. The returned data are then fetched into a DataFrame.

- SyntheticField for Custom Metrics: SyntheticField objects are created to calculate custom metrics, such as the `borrow-deposit ratio`, `user engagement metrics` for borrowed funds, and average values for borrow, deposit, and liquidation per user.

- Fetching Market Daily Snapshots: Queries are performed on the marketDailySnapshots entity of the subgraph, fetching specific market rates, converting timestamps, and restructuring the DataFrame using the pivot() function.

- Data Visualization: The data are plotted using plotly, showcasing interest rates over time and daily active users against datetime.

In conclusion, this code snippet effectively demonstrates how to utilize the Subgrounds library to interact with Ethereum subgraphs and process, manipulate, and visualize the extracted data. It's a useful guide for anyone looking to interact with Ethereum subgraphs and create custom metrics for data analysis.

About AAVE:

Aave is a decentralized finance protocol that allows people to lend and borrow cryptocurrencies. 

Lenders earn interest by depositing digital assets into specially created liquidity pools. 

Borrowers can then use their crypto as collateral to take out a flash loan using this liquidity.

Aave V3, introducing important new risk mitigation features and improved capital efficiency

V3 was designed with a flexible architecture for increased composability, and made it possible to build a variety of innovative features on top of the protocol such as new risk management tools that provide additional security and stability. 

Examples include Supply and Borrow Caps and Risk and Listing Admins. Aave V3 also improves capital efficiency and decentralized liquidity while lowering gas fees 25 percent.

In [2]:
## Playgrounds Gateway setup - visit app.playgrounds.network to get an api key to query decentralized subgraphs

# Import subgrounds
from subgrounds import Subgrounds

# Instantiate subgrounds and insert Playgrounds API key into header
sg = Subgrounds.from_pg_key("PGA_API_KEY")

# Insert desired subgraph id from decentralized subgraph. Find in subgraph url
aave_v3_id = "SUBGRAPH_ID"

In [3]:
# Load subgraph using playgrounds proxy endpoint 
aave_v3 = sg.load_subgraph(f"https://api.playgrounds.network/v1/proxy/subgraphs/id/{aave_v3_id}")

In [4]:
# How to query subgraphs with subgrounds
# Subgrounds allows the same arguments as you normally use on graphql

usage_metrics_daily = aave_v3.Query.usageMetricsDailySnapshots(
    first = 10,
    orderDirection = 'desc'
)

q_df_ex = sg.query_df([
    usage_metrics_daily.timestamp,
    usage_metrics_daily.dailyActiveBorrowers,
    usage_metrics_daily.dailyActiveDepositors,
])

In [5]:
q_df_ex

,usageMetricsDailySnapshots_timestamp,usageMetricsDailySnapshots_dailyActiveBorrowers,usageMetricsDailySnapshots_dailyActiveDepositors
0,1685517179,26,32
1,1685490923,48,64
2,1685404727,47,57
3,1685317403,53,69
4,1685231063,51,65
5,1685144063,44,74
6,1685057351,59,60
7,1684972751,59,81
8,1684885859,52,56
9,1684799987,37,51


In [6]:
from subgrounds.subgraph import SyntheticField
from datetime import datetime

usage_metrics_daily = aave_v3.UsageMetricsDailySnapshot

# Method 1
usage_metrics_daily.d_t = SyntheticField(
    f=lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_=SyntheticField.STRING,
    deps=usage_metrics_daily.timestamp,
)

# Method 2: This helper constructor makes it easy to convert timestamps into datetime objects.
usage_metrics_daily.datetime = SyntheticField.datetime_of_timestamp(usage_metrics_daily.timestamp)

usage_metrics_daily = aave_v3.Query.usageMetricsDailySnapshots(
    first = 100,
    orderDirection = 'desc'
)

sg.query_df([
    usage_metrics_daily.datetime,
    usage_metrics_daily.d_t,
    usage_metrics_daily.dailyActiveBorrowers,
    usage_metrics_daily.dailyActiveDepositors,
    usage_metrics_daily.dailyActiveLiquidatees,
    usage_metrics_daily.dailyActiveLiquidators,
])

,usageMetricsDailySnapshots_datetime,usageMetricsDailySnapshots_d_t,usageMetricsDailySnapshots_dailyActiveBorrowers,usageMetricsDailySnapshots_dailyActiveDepositors,usageMetricsDailySnapshots_dailyActiveLiquidatees,usageMetricsDailySnapshots_dailyActiveLiquidators
0,2023-05-31 02:12:59,2023-05-31 02:12:59,26,32,0,0
1,2023-05-30 18:55:23,2023-05-30 18:55:23,48,64,0,0
2,2023-05-29 18:58:47,2023-05-29 18:58:47,47,57,0,0
3,2023-05-28 18:43:23,2023-05-28 18:43:23,53,69,0,0
4,2023-05-27 18:44:23,2023-05-27 18:44:23,51,65,0,0
...,...,...,...,...,...,...
95,2023-02-25 17:58:23,2023-02-25 17:58:23,52,94,0,0
96,2023-02-24 16:59:35,2023-02-24 16:59:35,54,77,0,0
97,2023-02-23 17:40:47,2023-02-23 17:40:47,54,71,0,0
98,2023-02-22 17:44:23,2023-02-22 17:44:23,41,78,0,0


Let's play with synthetic fields some more and create some interesting transformations. That we can query as if regular GraphQL fields. 

SyntheticFields can created using the constructor, allowing for much more complex transformations.

In [7]:

# Query lending protocols entitty and fields from subgraph
# Here we are querying all of the fields in the lending protocols

aave_lending = aave_v3.Query.lendingProtocols()
aave_overview = sg.query_df(aave_lending)
aave_overview.squeeze()

lendingProtocols_id                                  0xbaa999ac55eace41ccae355c77809e68bb345170
lendingProtocols_name                                                                   Aave v3
lendingProtocols_slug                                                                   aave-v3
lendingProtocols_schemaVersion                                                            2.0.1
lendingProtocols_subgraphVersion                                                          1.1.0
lendingProtocols_methodologyVersion                                                       1.0.1
lendingProtocols_network                                                                MAINNET
lendingProtocols_type                                                                   LENDING
lendingProtocols_lendingType                                                             POOLED
lendingProtocols_riskType                                                              ISOLATED
lendingProtocols_cumulativeUniqueUsers  

In [8]:
# Query lending protocols entitty and specific fields 

aave_lending = aave_v3.Query.lendingProtocols()
aave_overview = sg.query_df([
    aave_lending.name,
    aave_lending.type,
    aave_lending.cumulativeBorrowUSD,
    aave_lending.cumulativeDepositUSD,
    aave_lending.cumulativeLiquidateUSD,
    aave_lending.cumulativePositionCount,
    aave_lending.cumulativeProtocolSideRevenueUSD,
    aave_lending.cumulativeSupplySideRevenueUSD,
    aave_lending.cumulativeTotalRevenueUSD,
    aave_lending.cumulativeUniqueUsers,
    aave_lending.cumulativeUniqueBorrowers,
    aave_lending.cumulativeUniqueDepositors,
    aave_lending.cumulativeUniqueLiquidatees,
    aave_lending.cumulativeUniqueLiquidators,
])
aave_overview.squeeze()

lendingProtocols_name                                          Aave v3
lendingProtocols_type                                          LENDING
lendingProtocols_cumulativeBorrowUSD                 3136236771.037705
lendingProtocols_cumulativeDepositUSD                4268369718.607094
lendingProtocols_cumulativeLiquidateUSD                  504376.008139
lendingProtocols_cumulativePositionCount                         19981
lendingProtocols_cumulativeProtocolSideRevenueUSD        457392.021437
lendingProtocols_cumulativeSupplySideRevenueUSD         3033007.248703
lendingProtocols_cumulativeTotalRevenueUSD               3490399.27014
lendingProtocols_cumulativeUniqueUsers                            7537
lendingProtocols_cumulativeUniqueBorrowers                        3637
lendingProtocols_cumulativeUniqueDepositors                       7428
lendingProtocols_cumulativeUniqueLiquidatees                        37
lendingProtocols_cumulativeUniqueLiquidators                        13
Name: 

In [9]:
from subgrounds.subgraph import SyntheticField

# Borrow_Deposit_Ratio: Calculate borrow deposit ratio by dividing the total borrow by the total deposit
aave_v3.LendingProtocol.borrow_dep_ratio = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.totalBorrowBalanceUSD,
    aave_v3.LendingProtocol.totalDepositBalanceUSD],
)

sg.query([aave_lending.totalBorrowBalanceUSD,
          aave_lending.totalDepositBalanceUSD,
          aave_lending.borrow_dep_ratio])

(559004617.9120322, 1463441773.4548209, 0.38197940502433647)

In [10]:
# User Engagement Metrics: You can calculate the proportion of users who have ever borrowed,
# by dividing each of the cumulativeUnique...
# fields by cumulativeUniqueUsers

aave_v3.LendingProtocol.user_engagement_borrowed = (
    abs(aave_v3.LendingProtocol.cumulativeUniqueBorrowers)
/abs(aave_v3.LendingProtocol.cumulativeUniqueUsers)
)

sg.query_df([aave_lending.cumulativeUniqueBorrowers,
             aave_lending.cumulativeUniqueUsers,
             aave_lending.user_engagement_borrowed])

,lendingProtocols_cumulativeUniqueBorrowers,lendingProtocols_cumulativeUniqueUsers,lendingProtocols_user_engagement_borrowed
0,3637,7537,0.482553


In [11]:
# Average Borrow, Deposit, and Liquidation Values: cumulativeBorrowUSD divided by cumulativeUniqueBorrowers,
# cumulativeDepositUSD divided by cumulativeUniqueDepositors, and cumulativeLiquidateUSD divided by 
# cumulativeUniqueLiquidatees will give you the average amount borrowed, deposited, and liquidated per user.

# Average borrow
aave_v3.LendingProtocol.avg_borrow_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeBorrowUSD,
    aave_v3.LendingProtocol.cumulativeUniqueBorrowers],
)

# Average deposit
aave_v3.LendingProtocol.avg_deposit_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeDepositUSD,
    aave_v3.LendingProtocol.cumulativeUniqueDepositors],
)

# Average liqudation per user
aave_v3.LendingProtocol.avg_liquidation_per_user = SyntheticField(
    lambda x, y: x/y,
    SyntheticField.FLOAT,
    [aave_v3.LendingProtocol.cumulativeLiquidateUSD,
    aave_v3.LendingProtocol.cumulativeUniqueLiquidatees],
)

sg.query_df([aave_lending.avg_borrow_per_user,
          aave_lending.avg_deposit_per_user,
          aave_lending.avg_liquidation_per_user])

,lendingProtocols_avg_borrow_per_user,lendingProtocols_avg_deposit_per_user,lendingProtocols_avg_liquidation_per_user
0,862314.207049,574632.433846,13631.784004


In [12]:
import pandas as pd
steth_market_id = '0x0b925ed163218f6662a35e0f0371ac234f9e9371'

snapshots = aave_v3.Query.marketDailySnapshots(
    first=100,
    orderBy=aave_v3.MarketDailySnapshot.timestamp,
    orderDirection='desc',
    where={
        'market':steth_market_id
    }
)

df = sg.query_df([
    snapshots.timestamp,
    snapshots.rates.rate,
    snapshots.rates.side,
    snapshots.rates.type
])

df['side_type'] = df['marketDailySnapshots_rates_side'] +'-' +  df['marketDailySnapshots_rates_type']
df2 = df[['marketDailySnapshots_timestamp', 'marketDailySnapshots_rates_rate', 'side_type']]
df2 = df2.pivot(index='marketDailySnapshots_timestamp',columns='side_type').droplevel(level=0, axis=1)
df2.index = pd.to_datetime(df2.index, unit='s')
df2

side_type,BORROWER-STABLE,BORROWER-VARIABLE,LENDER-VARIABLE
marketDailySnapshots_timestamp,,,
2023-02-21 23:59:11,7.856011,0.650513,0.022146
2023-02-22 23:07:11,7.815310,0.604724,0.018233
2023-02-23 23:40:47,7.840961,0.633582,0.020658
2023-02-24 22:51:47,7.862027,0.657281,0.022754
2023-02-25 23:58:23,7.837791,0.630015,0.020350
...,...,...,...
2023-05-27 08:24:59,7.523090,0.275977,0.000609
2023-05-28 18:39:11,7.543945,0.299438,0.001258
2023-05-29 22:31:59,7.526172,0.279444,0.000699


In [13]:
import plotly.express as px
fig = px.line(df2, x=df2.index, y="BORROWER-STABLE", title='STETH Interest Rates')
fig.show()

In [14]:
from subgrounds.contrib.plotly import Figure, Scatter

# Create the Scatter trace with appropriate field paths
trace = Scatter(
    x=usage_metrics_daily.datetime,
    y=usage_metrics_daily.dailyActiveUsers,
)

# Create the Figure instance with the trace and display it
fig = Figure(
    subgrounds=sg,
    traces=trace,
    layout=dict(
        title="Daily Active Users vs Datetime",
        xaxis=dict(title="Datetime"),
        yaxis=dict(title="Daily Active Users")
    ),
)
fig.figure.show()